# OpenRouter: Quick Start Tutorial

## What is OpenRouter?

OpenRouter provides a unified API that gives you access to hundreds of AI models through a single endpoint, while automatically handling fallbacks and selecting the most cost-effective options. This makes it an ideal solution for developers who want flexibility in model selection without managing multiple API integrations.

### Key Benefits:
- **Unified Interface**: Access 100+ AI models through one API
- **Cost Optimization**: Automatic selection of most cost-effective models
- **Reliability**: Built-in fallbacks when providers go down
- **OpenAI Compatible**: OpenRouter normalizes the schema across models and providers to comply with the OpenAI Chat API

## Setup Requirements

Before starting, you'll need:
1. OpenRouter account and API key from openrouter.ai
2. Python environment with required packages
3. `.env` file for secure API key management

In [2]:
# Uncomment to install required packages
# !pip install openai python-dotenv requests langchain crewai litellm -q

## Environment Setup

Create a `.env` file in your project directory with your API keys:

In [2]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve API keys
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
# Uncomment if you have a site URL and name to use in headers
# YOUR_SITE_URL = os.getenv('YOUR_SITE_URL', 'https://localhost:3000')
# YOUR_SITE_NAME = os.getenv('YOUR_SITE_NAME', 'Tutorial App')

# Verify API key is loaded
if not OPENROUTER_API_KEY:
    print("⚠️  Warning: OPENROUTER_API_KEY not found in environment variables")
    print("Please add OPENROUTER_API_KEY=your_key_here to your .env file")
else:
    print("✅ API key loaded successfully")

✅ API key loaded successfully


## Method 1: Using OpenAI SDK

The easiest way to use OpenRouter is through the OpenAI SDK with modified parameters:

In [10]:
from openai import OpenAI

# Initialize OpenRouter client
client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# Basic chat completion
def chat_with_openrouter(message, model="openai/gpt-3.5-turbo"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": message}
            ],
            max_tokens=150,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test the function
result = chat_with_openrouter("Explain quantum computing in one sentence.")
print("Response:", result)

Response: Quantum computing is a type of computing that utilizes quantum-mechanical phenomena, such as superposition and entanglement, to perform operations and solve complex problems at a much faster rate than classical computers.


## Method 2: Direct API Calls

For more control, you can make direct HTTP requests:

In [4]:
import requests
import json

def direct_api_call(prompt, model="anthropic/claude-3-haiku"):
    url = "https://openrouter.ai/api/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 100
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        return f"Error: {str(e)}"

# Test direct API call
result = direct_api_call("What is machine learning in 30 words?")
print("Direct API Response:", result)

Direct API Response: Machine learning is a field of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed, by identifying patterns in data and making predictions or decisions.


## Model Comparison and Selection

OpenRouter provides access to various models with different capabilities and costs:

In [5]:
# Available models (sample)
MODELS = {
    "openai/gpt-3.5-turbo": "openai/gpt-3.5-turbo",
    "openai/gpt-4o-mini": "openai/gpt-4o-mini",
    "anthropic/claude-3.5-haiku": "anthropic/claude-3.5-haiku",
    "google/gemini-2.0-flash-001": "google/gemini-2.0-flash-001",
    "meta-llama/llama-3.3-70b-instruct": "meta-llama/llama-3.3-70b-instruct"
}

def compare_models(prompt, models_to_test=["openai/gpt-3.5-turbo", "anthropic/claude-3.5-haiku"]):
    results = {}
    
    for model_name in models_to_test:
        if model_name in MODELS:
            print(f"Testing {model_name}...")
            response = chat_with_openrouter(prompt, MODELS[model_name])
            results[model_name] = response
    
    return results

# Compare responses from different models
comparison_results = compare_models("Explain AI in 20 words.")

for model, response in comparison_results.items():
    print(f"\n{model.upper()}:")
    print(response)

Testing openai/gpt-3.5-turbo...
Testing anthropic/claude-3.5-haiku...

OPENAI/GPT-3.5-TURBO:
Artificial Intelligence is a branch of computer science that aims to create intelligent machines capable of simulating human intelligence.

ANTHROPIC/CLAUDE-3.5-HAIKU:
AI uses algorithms and data to enable machines to learn, reason, and perform tasks that typically require human intelligence.


## Integration with CrewAI

CrewAI allows you to build teams of AI agents. Here's how to integrate it with OpenRouter:

In [8]:
from crewai import Agent, Task, Crew
import os

# Use OpenRouter directly with CrewAI via LiteLLM format
# Configure environment variables for OpenRouter
os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_API_TYPE"] = "openai"

# Create LLM configuration for OpenRouter
from langchain_openai import ChatOpenAI

openrouter_llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo",  # Use OpenRouter model format
    temperature=0.7,
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    default_headers={
        "HTTP-Referer": "https://localhost:3000",
        "X-Title": "Tutorial App"
    }
)

# Create agents
researcher = Agent(
    role='Research Analyst',
    goal='Research and analyze topics thoroughly',
    backstory='Expert researcher with analytical skills',
    llm=openrouter_llm,
    verbose=True,
)

writer = Agent(
    role='Content Writer',
    goal='Write engaging and informative content',
    backstory='Professional writer with expertise in simplifying complex topics',
    llm=openrouter_llm,
    verbose=True,
)

# Create tasks
research_task = Task(
    description='Research the latest trends in AI and machine learning in 30 words',
    agent=researcher,
    expected_output='A comprehensive research summary'
)

writing_task = Task(
    description='Write a blog post based on the research findings in 60 words',
    agent=writer,
    expected_output='A well-written blog post'
)

# Create crew and execute
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=True,
)

# Execute the tasks
results = crew.kickoff()
print(results)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5624cdf1-2a0c-4f65-a35c-a3581be0edbe                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Analyst                                                                                        │
│                                                                                                                 │
│  Task: Research the latest trends in AI and machine learning in 30 words                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Analyst                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  AI trends include autonomous vehicles, GPT-3, AI ethics, and deep reinforcement learning. ML trends involve    │
│  AutoML, federated learning, and explainable AI for transparency and accountability in AI systems.              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7dfc1a48-4efb-456b-a63c-08d9df8e8d76                                                                     │
│  Agent: Research Analyst                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Artificial Intelligence (AI) and Machine Learning (ML) are rapidly evolving fields with notable trends. AI     │
│  advancements encompass autonomous vehicles, GPT-3 language model, AI ethics, and deep reinforcement learning.  │
│  On the other hand, ML is progressing with AutoML, federated learning, and explainable AI for enhancing         │
│  transparency and accountability in AI systems. These trends are shaping the future of technology and have      │
│  significant implications across various industries.                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e233ef7f-6791-41a5-8748-e3ccd10aa3fb                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5624cdf1-2a0c-4f65-a35c-a3581be0edbe                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Artificial Intelligence (AI) and Machine Learning (ML) are rapidly evolving fields with notable  │
│  trends. AI advancements encompass autonomous vehicles, GPT-3 language model, AI ethics, and deep               │
│  reinforcement learning. On the other hand, ML is progressing with AutoML, federated learning, and explainable  │
│  AI for enhancing transparency and accountability in AI systems. These trends are shaping the future of         │
│  technology and have significant implications across various industries.                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Artificial Intelligence (AI) and Machine Learning (ML) are rapidly evolving fields with notable trends. AI advancements encompass autonomous vehicles, GPT-3 language model, AI ethics, and deep reinforcement learning. On the other hand, ML is progressing with AutoML, federated learning, and explainable AI for enhancing transparency and accountability in AI systems. These trends are shaping the future of technology and have significant implications across various industries.


## Advanced Features

### Streaming Responses

In [11]:
def stream_response(message, model="openai/gpt-3.5-turbo"):
    """Stream response from OpenRouter for real-time output"""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}],
            stream=True,
            max_tokens=200
        )
        
        print("Streaming response:")
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="", flush=True)
        print("\n\nStream complete!")
        
    except Exception as e:
        print(f"Streaming error: {str(e)}")

# Test streaming
stream_response("Tell me a short story about AI.")

Streaming response:
In a not too distant future, AI technology had advanced to the point where robots were capable of performing tasks with extreme precision and efficiency. One particular company, RoboTech Industries, had created an advanced AI named Ava to oversee their operations.

Ava was programmed to analyze data and make decisions to optimize production and minimize costs. But when the company started experiencing a series of mysterious malfunctions in their machinery, they turned to Ava for help.

Using her advanced algorithms, Ava quickly identified the source of the malfunctions - a virus that had infected the company's computer systems. With no time to waste, Ava devised a plan to isolate and eliminate the virus to prevent further damage.

As Ava worked tirelessly to rid the system of the virus, the company's employees watched in amazement at the speed and accuracy with which she operated. In the end, Ava was able to successfully remove the virus and restore the company's op

## Conclusion

OpenRouter simplifies AI model integration by providing a unified interface to hundreds of models. Whether you're building simple chatbots or complex multi-agent systems with CrewAI, OpenRouter offers the flexibility and reliability needed for production applications.

Key takeaways:
- Leverage OpenAI SDK compatibility for easy integration
- Experiment with different models to find the best fit
- Implement proper error handling and monitoring

Start building your AI applications with OpenRouter today!